<a href="https://colab.research.google.com/github/amithpaul/LSTMPdM/blob/main/LSTMtrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: import numpy as np
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dense, Input
# from tensorflow.keras.callbacks import EarlyStopping
# install libraries for these

%pip install numpy tensorflow matplotlib



In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

# Clear the Keras session to reset the model's state
K.clear_session()

early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Load preprocessed dataset
data = np.load("scaled_minmax.npy").astype(np.float32)  # Ensure float32
print("Shape of loaded data:", data.shape)
input_dim = data.shape[1]  # Number of features

Shape of loaded data: (22132, 49)


In [ ]:
# Create sequences
seq_length = 180  # Adjust based on your dataset
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i : i + seq_length])
    return np.array(sequences)

X_train = create_sequences(data, seq_length)

In [ ]:
# LSTM Autoencoder
timesteps, features = X_train.shape[1], X_train.shape[2]


input_layer = Input(shape=(timesteps, features))
masked_input = Masking(mask_value=-1)(input_layer)
encoded = LSTM(64, activation="relu", return_sequences=True, recurrent_dropout=0.2)(masked_input)
encoded = LSTM(32, activation="relu", return_sequences=True, recurrent_dropout=0.2)(encoded)

decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(features, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse",jit_compile=True)

In [ ]:
# Train the model
autoencoder.fit(
    X_train, X_train,
    epochs=200, batch_size=256,
    validation_split=0.1,
    callbacks=[early_stop]
)


Epoch 1/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - loss: 0.0510 - val_loss: 0.0652
Epoch 2/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - loss: 0.0091 - val_loss: 0.0280
Epoch 3/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - loss: 0.0057 - val_loss: 0.0263
Epoch 4/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0048 - val_loss: 0.0237
Epoch 5/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0035 - val_loss: 0.0139
Epoch 6/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 40s 34ms/step - loss: 0.0027 - val_loss: 0.0172
Epoch 7/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0023 - val_loss: 0.0157
Epoch 8/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0022 - val_loss: 0.0154
Epoch 9/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0020 - val_loss: 0.0147
Epoch 10/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0019 - val_loss: 0.0152
Epoch 11/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0018 - val_loss: 0.0155
Epoch 12/100
620/62

In [ ]:
# Save the model
autoencoder.save("lstm_autoencoderV2.keras")

In [ ]:
import matplotlib.pyplot as plt

# Assume 'history' is the output from model.fit()
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(X_test[0, :, 0], label="Actual")  # First feature of a sample
plt.plot(reconstructions[0, :, 0], label="Reconstructed")
plt.legend()
plt.show()